In [136]:
import pandas as pd
import numpy as np
from data_extraction.dummy_data_extractor import extract_dummy_data
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
# imports for DOT and seasonalNaive models
import re
from statsforecast.models import (
    # HoltWinters,
    # CrostonClassic as Croston, 
    # HistoricAverage,
    DynamicOptimizedTheta as DOT,
    SeasonalNaive,
    # AutoARIMA
)
from statsforecast import StatsForecast
# imports for polynomialRegression model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
# imports for randomForestRegressor model
from sklearn.ensemble import RandomForestRegressor
# imports for kNeighborsRegressor model
from sklearn.neighbors import KNeighborsRegressor


# other imports
from prophet import Prophet


In [137]:
master_table = extract_dummy_data("dummy_data")
master_table

,pointID,unit,dqType,dqStart,dqDuration,pointInterval,his
0,@p:dmc_All:r:2e223a3a-5053d2b0 DMC Building 1 ...,°C,Nulls,2023-05-09 23:55:00+04:00,2 days 00:05:00,0 days 00:05:00,Timestamp val \ 0...


In [138]:
# #---------
# # TESTING for running a single model
# #---------

# # Testing variables
# row = master_table.iloc[0]
# df = row["his"]
# df.set_index(df.columns[0], inplace=True, drop=True)
# length_of_missing_data = row["dqDuration"]
# data_logging_interval = row["pointInterval"]
# dqStart	= row['dqStart']
# dqDuration = row['dqDuration']
# #dqStart = dqStart - length_of_missing_data


### Models

###### Working Models

In [139]:
def seasonal_naive(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Inputs
    df: df used for training set (from SS)
    length_of_missing_data: interval length of missing data (from SS)
    data_logging_interval: data logging interval - called from the hisDQInterval tag on the point (from SS)

    Output
    forecasts_df: dataframe with predictions for the period missing data. Index names as ts, values column named as "v0
    """
    
    
    # step 1 convert the grid to a dataframe, and set first column as index     ### UNCOMMENT THIS ONLY IF RUNNING THE MODEL DIRECTLY ON SS. THIS IS DONE IN THE ENSEMBLE MODEL SO NO NEED TO HAVE THIS WHEN RUNNING THROUGH ENSEMBLE MODEL
    #df = df.to_dataframe()
    #df.set_index(df.columns[0], inplace=True, drop=True)

    # rename the first column as "target"
    new_column_name = "target"
    df = df.rename(columns={df.columns[0]: new_column_name})

    # keep only the history BEFORE the start of the data quality issue, since this is a statisitcal model not ML model
    df = df[df.index < dqStart]

    # format the df to statsforecast format
    df = df.reset_index()
    df = df.rename(columns={df.columns[0]: 'ds', df.columns[1]: "y"})
    df['unique_id'] = "v0"    

    # number of predictions
    horizon = int(length_of_missing_data/data_logging_interval) #+ 1 # why -1? because if you do length_of_missing_data/data_logging_interval you will get prediction length that is exclusive of the start ts (start ts is the last ts with actual data before the gap), and inclusive of the end ts (end ts is the first ts with actual data after the gap). +1 to get predictions also for the start and end timestamp. Can remove them later

    # season length
    season_length = int(pd.Timedelta(24, 'h') / data_logging_interval)      

    # frequency
    freq = str(data_logging_interval.total_seconds()/3600)+"h"


    # LIST OF MODELS
    models = [
        SeasonalNaive(season_length=season_length) 
    ]

    # The Model
    sf = StatsForecast( 
        models=models,
        freq=freq, 
        # fallback_model = SeasonalNaive(season_length=season_length),
        n_jobs=-1,
    )

    # Model fitting
    forecasts_df = sf.forecast(df=df[["ds", "y", "unique_id"]], h=horizon, level=[90])  

    # removing the -hi- and -lo- columns
    for col in forecasts_df.columns:
        if re.search("-hi-", col) or re.search("-lo-", col):
            forecasts_df.drop(col, axis=1, inplace=True)
            
    forecasts_df = forecasts_df.rename(columns={"ds": "timestamp", "SeasonalNaive":"seasonalNaive"})

    forecasts_df.set_index("timestamp", inplace=True)

    return forecasts_df

In [140]:
def dynamic_optimized_theta(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Inputs
    df: df used for training set (from SS)
    length_of_missing_data: interval length of missing data (from SS)
    data_logging_interval: data logging interval - called from the hisDQInterval tag on the point (from SS)

    Output
    forecasts_df: dataframe with predictions for the period missing data. Index names as ts, values column named as "v0
    """
    

    # step 1 convert the grid to a dataframe, and set first column as index     ### UNCOMMENT THIS ONLY IF RUNNING THE MODEL DIRECTLY ON SS. THIS IS DONE IN THE ENSEMBLE MODEL SO NO NEED TO HAVE THIS WHEN RUNNING THROUGH ENSEMBLE MODEL
    #df = df.to_dataframe()
    #df.set_index(df.columns[0], inplace=True, drop=True)

    # rename the first column as "target"
    new_column_name = "target"
    df = df.rename(columns={df.columns[0]: new_column_name})

    # keep only the history BEFORE the start of the data quality issue, since this is a statisitcal model not ML model
    df = df[df.index < dqStart]

    # format the df to statsforecast format
    df = df.reset_index()
    df = df.rename(columns={df.columns[0]: 'ds', df.columns[1]: "y"})
    df['unique_id'] = "v0"    

    # number of predictions
    horizon = int(length_of_missing_data/data_logging_interval) #+ 1 # why -1? because if you do length_of_missing_data/data_logging_interval you will get prediction length that is exclusive of the start ts (start ts is the last ts with actual data before the gap), and inclusive of the end ts (end ts is the first ts with actual data after the gap). +1 to get predictions also for the start and end timestamp. Can remove them later

    # season length
    season_length = int(pd.Timedelta(24, 'h') / data_logging_interval)      

    # frequency
    freq = str(data_logging_interval.total_seconds()/3600)+"h"


    # LIST OF MODELS
    models = [
        DOT(season_length=season_length) 
    ]

    # The Model
    sf = StatsForecast( 
        models=models,
        freq=freq, 
        # fallback_model = SeasonalNaive(season_length=season_length),
        n_jobs=-1,
    )

    # Model fitting
    forecasts_df = sf.forecast(df=df[["ds", "y", "unique_id"]], h=horizon, level=[90])  

    # removing the -hi- and -lo- columns
    for col in forecasts_df.columns:
        if re.search("-hi-", col) or re.search("-lo-", col):
            forecasts_df.drop(col, axis=1, inplace=True)
            
    forecasts_df = forecasts_df.rename(columns={"ds": "timestamp", "DynamicOptimizedTheta":"dynamicOptimizedTheta"})

    forecasts_df.set_index("timestamp", inplace=True)

    return forecasts_df

In [141]:
def polynomial_regression(df, length_of_missing_data, data_logging_interval, dqStart, featureNumber):

    """
    Inputs
    df: df used for training set (from SS)
    dqStart: start of the predictions

    Output
    forecasts_df: dataframe with predictions for the period missing data. Index names as ts
    """

    # Drop all NaN
    # df = df.dropna()

    # Splitting variables
    y = df[df.columns[0]]  # independent variable
    X = df[[df.columns[featureNumber+1]]]  # dependent variable

    # Filter data for training and testing
    X_train = X[X.index < dqStart]
    y_train = y[X.index < dqStart]
    X_test = X[X.index >= dqStart]
    #y_test = y[X.index >= dqStart]

    # Generate polynomial features
    poly = PolynomialFeatures(degree = 4)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Train polynomial regression model on the whole dataset
    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    y_pred = model.predict(X_test_poly)

    # Create a new DataFrame with the timestamp as index and y_pred as values
    pred_df = pd.DataFrame(data=y_pred, index=X_test.index, columns=['y_pred'])

    return pred_df

In [142]:
def random_Forest_Regressor(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Input
    master_table: main table received from SS

    Output
    df: dataframe with predictions for all rows with missing columns. Index names as ts
    """
    X = df.iloc[:,1:-1]
    y = df.iloc[:,0:1]  

    X_train = X[X.index < dqStart]
    X_test = X[X.index >= dqStart]
    y_train = y[y.index < dqStart]

    # Fitting Random Forest Regression to the dataset
    regressor = RandomForestRegressor(n_estimators=10, random_state=0, oob_score=True)
    
    # Fit the regressor with x and y data
    regressor.fit(X_train, y_train)
    pred = regressor.predict(X_test)
    predictions = pd.DataFrame(data=pred, index=X_test.index, columns=['y_pred'])
    
    return predictions

In [143]:
def kNeighbors_Regressor_Uniform(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Input
    df:data table from SS with the ts as index
    dqStart: start datetime

    Output
    df: dataframe with predictions for all rows with missing data inclusive of the start date and end date. Index names as ts

    #Uniform: gives each data point equal weight
    """
    X = df.iloc[:,1:-1]
    y = df.iloc[:,0:1]  

    X_train = X[X.index < dqStart]
    X_test = X[X.index >= dqStart]
    y_train = y[y.index < dqStart]

    knn_regressor = KNeighborsRegressor(n_neighbors=3,weights="uniform")
    knn_regressor.fit(X_train, y_train)
    pred = knn_regressor.predict(X_test)
    predictions = pd.DataFrame(data=pred, index=X_test.index, columns=['y_pred'])

    return predictions

###### Models to be integrated

In [144]:
def facebook_pred(df, length_of_missing_data, data_logging_interval, dqStart):
    
    df.reset_index(inplace=True)
    df = df.dropna()

    # Keep only the first two columns
    df = df.iloc[:, :2]

    # # renaming columns
    df.columns = ['ds', 'temp']

    # Remove ' Dubai' from the datetime strings
    df['ds'] = df['ds'].str.replace(' Dubai', '', regex=False)

    # Try converting the 'ds' column to datetime format with error handling
    try:
        df['ds'] = pd.to_datetime(df['ds'], format="%Y-%m-%dT%H:%M:%S%z")
    except ValueError as e:
        print(f"Error parsing datetime: {e}")
        print("Some datetime strings could not be parsed. Check your data.")
        problematic_rows = df[pd.to_datetime(df['ds'], format="%Y-%m-%dT%H:%M:%S%z", errors='coerce').isna()]
        print("Problematic rows:")
        print(problematic_rows)
        return None

    # Drop rows where datetime parsing failed
    df = df.dropna(subset=['ds'])

    # Clean temperature column and convert to numeric
    df['temp'] = df['temp'].str.replace('°C', '').astype(float)

    # Rename columns for convenience
    df.columns = ['ds', 'y']

    # Separate data for temperature
    df_temp = df.copy()

    # Ensure 'ds' column is timezone-naive
    # df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)

    # Initialize Prophet models with tuned hyperparameters
    model_temp = Prophet(seasonality_mode='additive',     # Adjust based on data exploration
                         interval_width=0.95,              # Adjust prediction interval if needed
                         changepoint_prior_scale=0.01)    # Tune based on data patterns

    # Fit the models
    model_temp.fit(df_temp)

    # Calculate number of predictions
    samples = int(length_of_missing_data / data_logging_interval) + 1

    # Specify the start date for prediction
    # dq_start = pd.Timestamp(dqStart, tz='Asia/Dubai').tz_localize(None)
    dq_start = dqStart

    # Create future DataFrame starting from dq_start
    future_temp = model_temp.make_future_dataframe(periods=samples, freq='5T')

    # Adjust 'ds' column to start from dq_start
    future_temp['ds'] = dq_start + pd.to_timedelta(range(len(future_temp)), unit='m')

    # Ensure 'ds' column is timezone-naive
    future_temp['ds'] = future_temp['ds'].dt.tz_localize(None)

    # Predict the future values
    forecast_temp = model_temp.predict(future_temp)

    # Ensure 'ds' column in forecast_temp is timezone-naive
    forecast_temp['ds'] = forecast_temp['ds'].dt.tz_localize(None)

    # Convert dq_start to timezone-naive
    dq_start = dq_start.tz_localize(None)

    # Filter predictions to start from dq_start
    predictions = forecast_temp[forecast_temp['ds'] >= dq_start][['ds', 'yhat']]

    return predictions


In [145]:
def xgboost_1(df, length_of_missing_data, data_logging_interval, dqStart):
    df.reset_index(inplace=True)
    df = df.dropna()

    # Keep only the first two columns
    df = df.iloc[:, :2]

    # Rename columns
    df.columns = ['ds', 'temp']

    # Remove ' Dubai' from the datetime strings
    df['ds'] = df['ds'].astype(str).str.replace(' Dubai', '', regex=False)

    # Convert the 'ds' column to datetime format
    df['ds'] = pd.to_datetime(df['ds'], format="%Y-%m-%dT%H:%M:%S%z")

    # Drop rows where datetime parsing failed
    df = df.dropna(subset=['ds'])

    # Clean temperature column and convert to numeric
    df['temp'] = df['temp'].str.replace('°C', '').astype(float)

    # Rename columns for convenience
    df.columns = ['ds', 'y']

    # Ensure 'ds' column is timezone-naive
    df['ds'] = df['ds'].dt.tz_localize(None)

    # Extract numerical features from datetime
    df['year'] = df['ds'].dt.year
    df['month'] = df['ds'].dt.month
    df['day'] = df['ds'].dt.day
    df['hour'] = df['ds'].dt.hour
    df['minute'] = df['ds'].dt.minute

    # Create future DataFrame starting from dq_start
    future_periods = int(length_of_missing_data / data_logging_interval) + 1
    dq_start = pd.Timestamp(dqStart).tz_convert('Asia/Dubai').tz_localize(None)
    future_temp = pd.DataFrame()
    future_temp['ds'] = [dq_start + timedelta(minutes=5 * i) for i in range(future_periods)]

    # Extract features for XGBoost
    future_temp['year'] = future_temp['ds'].dt.year
    future_temp['month'] = future_temp['ds'].dt.month
    future_temp['day'] = future_temp['ds'].dt.day
    future_temp['hour'] = future_temp['ds'].dt.hour
    future_temp['minute'] = future_temp['ds'].dt.minute

    # Initialize XGBoost model
    model_temp = xgb.XGBRegressor()

    # Fit the model
    model_temp.fit(df[['year', 'month', 'day', 'hour', 'minute']], df['y'])

    # Predict the future values
    future_temp['yhat'] = model_temp.predict(future_temp[['year', 'month', 'day', 'hour', 'minute']])

    # Filter predictions to start from dq_start
    predictions = future_temp[['ds', 'yhat']]

    # Set 'ds' as the index
    predictions.set_index('ds', inplace=True)

    return predictions

In [146]:
def xgboost_2(df, length_of_missing_data, data_logging_interval, dqStart):
    df.reset_index(inplace=True)
    df = df.dropna()

    # Keep only the first two columns
    df = df.iloc[:, :2]

    # Rename columns
    df.columns = ['ds', 'temp']

    # Remove ' Dubai' from the datetime strings
    df['ds'] = df['ds'].astype(str).str.replace(' Dubai', '', regex=False)

    # Convert the 'ds' column to datetime format
    df['ds'] = pd.to_datetime(df['ds'], format="%Y-%m-%dT%H:%M:%S%z")

    # Drop rows where datetime parsing failed
    df = df.dropna(subset=['ds'])

    # Clean temperature column and convert to numeric
    df['temp'] = df['temp'].str.replace('°C', '').astype(float)

    # Rename columns for convenience
    df.columns = ['ds', 'y']

    # Ensure 'ds' column is timezone-naive
    dq_start = pd.Timestamp(dqStart).tz_convert('Asia/Dubai').tz_localize(None)

    # Extract numerical features from datetime
    df['year'] = df['ds'].dt.year
    df['month'] = df['ds'].dt.month
    df['day'] = df['ds'].dt.day
    df['hour'] = df['ds'].dt.hour
    df['minute'] = df['ds'].dt.minute

    # Create future DataFrame starting from dq_start
    future_periods = int(length_of_missing_data / data_logging_interval) + 1
    dq_start = pd.Timestamp(dqStart, tz='Asia/Dubai').tz_localize(None)
    future_temp = pd.DataFrame()
    future_temp['ds'] = [dq_start + timedelta(minutes=5 * i) for i in range(future_periods)]

    # Extract features for XGBoost
    future_temp['year'] = future_temp['ds'].dt.year
    future_temp['month'] = future_temp['ds'].dt.month
    future_temp['day'] = future_temp['ds'].dt.day
    future_temp['hour'] = future_temp['ds'].dt.hour
    future_temp['minute'] = future_temp['ds'].dt.minute

    ## Initialize XGBoost model with parameters to reduce noise
    model_temp = xgb.XGBRegressor(
        n_estimators=100,   # Number of boosting rounds
        max_depth=3,        # Maximum depth of each tree
        learning_rate=0.1,  # Learning rate
        min_child_weight=1, # Minimum sum of instance weight needed in a child
        subsample=0.8,      # Subsample ratio of the training instances
        colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree
        objective='reg:squarederror'  # Objective function for regression task
    )

    # Fit the model
    model_temp.fit(df[['year', 'month', 'day', 'hour', 'minute']], df['y'])

    # Predict the future values
    future_temp['yhat'] = model_temp.predict(future_temp[['year', 'month', 'day', 'hour', 'minute']])

    # Filter predictions to start from dq_start
    predictions = future_temp[['ds', 'yhat']]

    # Set 'ds' as the index
    predictions.set_index('ds', inplace=True)

    return predictions

###### Ensemble Model

In [147]:
def ensemble_model(python_master_table, filter=True):
    """
    Function to run all models, and return the one with lowest RMSE.
    Models running through the ensemble model will have input DataFrame (AKA the "his" column on master_table) 
    with timestamp as index, target variable as first column, feature variables as the rest of the columns.

    Make sure the output predictions of all models are INCLUSIVE of both the "start ts" and "end ts" (AKA
    last ts with real data before gap, and first ts with real data after gap) 

    Make sure to follow camelCase for DataFrame column naming for compatibility with SS

    filter = True. Parameter to show all predictions for each dq issue, or only ones filtered with rmse and mape.
    """


    # dictionary to save predictions for each point
    scores_df_dict = {
    "pointID": [],
    "predictions": [],
    "rmse": [],
    "mape": [],
    "modelName": []
    }
    # Create a DataFrame from the dictionary
    scores_df = pd.DataFrame(scores_df_dict)


    for i, row in python_master_table.iterrows():

        #-----------------
        # INPUTS TO MODELS
        #-----------------
        # point id on SS
        pointID = row["pointID"]
        # data logging interval of the point 
        data_logging_interval = row["pointInterval"]
        freq = str(data_logging_interval.total_seconds()/3600)+"h"
        # the history df containing training data for the models
        df = row["his"]#.to_dataframe()                                      #### IMPORTANT : UNCOMMENT THIS ON SS  @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        df.set_index(df.columns[0], inplace=True, drop=True)
        df = df.asfreq(freq)
        # dqStart and duration of dq issue (dqEnd = dqStart+length_of_missing_data)
        dqStart = row["dqStart"]
        length_of_missing_data = row["dqDuration"]


        # ----------------------
        # Training/Testing sets 
        # ----------------------
        data_before_gap = df.loc[:dqStart][:-1] # :-1 to not include the first ts with nulls, since df.loc[:dqStart] will include the first ts with NAN
        # doing a bfill and ffill to make sure there are no nulls in the training data. This step should be taken care of on SS by interpolating. This is done as a cautionary measure as nulls in the data will cause some models to fail
        data_before_gap.bfill(inplace=True)
        data_before_gap.ffill(inplace=True)
        train_data = data_before_gap.loc[:dqStart-length_of_missing_data][:-1]
        test_data = data_before_gap.loc[dqStart-length_of_missing_data:dqStart]
        # test_data timestamps used to slice the predictions_for_rmse df to have exact dimension as testing set. This prevents raising error due to mismatching lengths when using the rmse or mape functions
        test_data_timestamps = pd.date_range(start=test_data.index[0], end=test_data.index[-1], freq=test_data.index.freq)


        # ----------------------------
        # Timestamps for null duration
        # ----------------------------
        start = row['dqStart']
        duration = row['dqDuration']
        interval = row['pointInterval']
        timestamps = pd.date_range(start=start, end=start+duration, freq=interval)[1:-1] # clipping the first and last timestamps, as they already exist with actual data on SS


        #----------------------------
        # Dict of Data Quality Models                              ############# ADD NEW MODELS HERE ################
        #----------------------------

        # UNIVARIATE Models
        dq_models_univariate = {
            "Seasonal Naive" : seasonal_naive,
            "Dynamic Optimized Theta": dynamic_optimized_theta
        }

        # MULTIVARIATE Models using one feature at a time
        dq_models_multivariate_1feature = {
            "Polynomial Regression" : polynomial_regression,
        }

        # MULTIVARIATE Models using all features to predict target
        dq_models_multivariate = {
            "Random Forest Regressor" : random_Forest_Regressor,
            "KNN Regressor Uniform ": kNeighbors_Regressor_Uniform,
            # "XGBoost 1": xgboost_1,
            # "XGboost 2": xgboost_2,
            # "XGboost 3": xgboost_3
        }


        ############################
        # LOOP FOR UNIVARIATE MODELS
        ############################
        for model_name, model in dq_models_univariate.items():

            #------------------------
            # ** Calculating RMSE **
            #------------------------
            # the prediction. USED ONLY TO EVALUATE RMSE
            predictions_for_rmse = model(train_data, length_of_missing_data, data_logging_interval, dqStart)
            predictions_for_rmse = predictions_for_rmse[predictions_for_rmse.index.isin(test_data_timestamps)]
            rmse_score = mean_squared_error(test_data[test_data.columns[0]].to_numpy(), predictions_for_rmse[predictions_for_rmse.columns[0]].to_numpy(), squared=False)
            mape_score = mean_absolute_percentage_error(test_data[test_data.columns[0]].to_numpy(), predictions_for_rmse[predictions_for_rmse.columns[0]].to_numpy())


            #------------------
            # ** Predictions **
            #------------------
            # the predictions. USED FOR DATA CLEANING (uses all the data as training)
            predictions_for_data_quality = model(df, length_of_missing_data, data_logging_interval, dqStart)

            # keep only timestamps for null periods (rows where there are null values on SS)
            predictions_for_data_quality = predictions_for_data_quality[predictions_for_data_quality.index.isin(timestamps)]

            # reset index to make the ts a column instead of index. SS doesnt show the index of a DF
            predictions_for_data_quality = predictions_for_data_quality.reset_index()

            # rename the ts and predictions column to "ts" and "predictions", to have similar naming for all ouutputs of models (makes it easier as well when using the dcInsert function on SS.)
            predictions_for_data_quality.columns = ["ts", "predictions"]

            # append data to the scores DF
            row_to_append = {'pointID': pointID, 'predictions': predictions_for_data_quality, 
                                "rmse": rmse_score, "mape": mape_score,
                                "modelName": model_name, 
                                "identifier": 
                                    str(row["pointID"])
                                    +str(row["dqStart"])
                                    +str(row["dqDuration"])
                                    +str(row["dqType"])}

            scores_df = pd.concat([scores_df, pd.DataFrame([row_to_append])], ignore_index=True)


        ##############################
        # Loop for MULTIVARIATE MODELS
        ##############################
        if len(df.columns)>1:  # only run multivariate if there are features available to use from the master table
            
            ##############
            # Multivariate models where all features are used to predict the target
            for model_name, model in dq_models_multivariate.items():
                #------------------------
                # ** Calculating RMSE **
                #------------------------
                # the prediction. USED ONLY TO EVALUATE RMSE
                predictions_for_rmse = model(data_before_gap, length_of_missing_data, data_logging_interval, dqStart-length_of_missing_data)
                predictions_for_rmse = predictions_for_rmse[predictions_for_rmse.index.isin(test_data_timestamps)]  
                rmse_score = mean_squared_error(test_data[test_data.columns[0]].to_numpy(), predictions_for_rmse[predictions_for_rmse.columns[0]].to_numpy(), squared=False)
                mape_score = mean_absolute_percentage_error(test_data[test_data.columns[0]].to_numpy(), predictions_for_rmse[predictions_for_rmse.columns[0]].to_numpy())

                #------------------
                # ** Predictions **
                #------------------
                # the predictions. USED FOR DATA CLEANING (uses all the data as training)
                predictions_for_data_quality = model(df, length_of_missing_data, data_logging_interval, dqStart)

                # keep only timestamps for null periods (rows where there are null values on SS)
                predictions_for_data_quality = predictions_for_data_quality[predictions_for_data_quality.index.isin(timestamps)]   # UNCOMMENT THIS ONCE THE interval of the data column in SS master table is corrected

                # reset index to make the ts a column instead of index. SS doesnt show the index of a DF
                predictions_for_data_quality = predictions_for_data_quality.reset_index()

                # rename the ts and predictions column to "ts" and "predictions", to have similar naming for all ouutputs of models (makes it easier as well when using the dcInsert function on SS.)
                predictions_for_data_quality.columns = ["ts", "predictions"]

                # append data to the scores DF
                row_to_append = {'pointID': pointID, 'predictions': predictions_for_data_quality, 
                                "rmse": rmse_score, "mape": mape_score,
                                "modelName": model_name, 
                                "identifier": 
                                    str(row["pointID"])
                                    +str(row["dqStart"])
                                    +str(row["dqDuration"])
                                    +str(row["dqType"])}

                scores_df = pd.concat([scores_df, pd.DataFrame([row_to_append])], ignore_index=True)

            ##############
            # Multivariate models where 1 feature is used at a time to predict the target
            for model_name, model in dq_models_multivariate_1feature.items():
                    
                    # Loop over different features to be used one at a time
                    for featureNumber, featureName in enumerate(df.columns.tolist()[1:]):

                        #------------------------
                        # ** Calculating RMSE **
                        #------------------------
                        # the prediction. USED ONLY TO EVALUATE RMSE
                        predictions_for_rmse = model(data_before_gap, length_of_missing_data, data_logging_interval, dqStart-length_of_missing_data, featureNumber)
                        predictions_for_rmse = predictions_for_rmse[predictions_for_rmse.index.isin(test_data_timestamps)]  
                        rmse_score = mean_squared_error(test_data[test_data.columns[0]].to_numpy(), predictions_for_rmse[predictions_for_rmse.columns[0]].to_numpy(), squared=False)
                        mape_score = mean_absolute_percentage_error(test_data[test_data.columns[0]].to_numpy(), predictions_for_rmse[predictions_for_rmse.columns[0]].to_numpy())

                        #------------------
                        # ** Predictions **
                        #------------------
                        # the predictions. USED FOR DATA CLEANING (uses all the data as training)
                        predictions_for_data_quality = model(df, length_of_missing_data, data_logging_interval, dqStart, featureNumber)

                        # keep only timestamps for null periods (rows where there are null values on SS)
                        predictions_for_data_quality = predictions_for_data_quality[predictions_for_data_quality.index.isin(timestamps)]

                        # reset index to make the ts a column instead of index. SS doesnt show the index of a DF
                        predictions_for_data_quality = predictions_for_data_quality.reset_index()

                        # rename the ts and predictions column to "ts" and "predictions", to have similar naming for all ouutputs of models (makes it easier as well when using the dcInsert function on SS.)
                        predictions_for_data_quality.columns = ["ts", "predictions"]

                        # append data to the scores DF
                        row_to_append = {'pointID': pointID, 'predictions': predictions_for_data_quality, 
                                        "rmse": rmse_score, "mape": mape_score,
                                        "modelName": model_name+" - Feature: " + str(featureName), 
                                        "identifier": 
                                            str(row["pointID"])
                                            +str(row["dqStart"])
                                            +str(row["dqDuration"])
                                            +str(row["dqType"])}

                        scores_df = pd.concat([scores_df, pd.DataFrame([row_to_append])], ignore_index=True)

    # filtering on RMSE and MAPE
    if filter:
        # keep only predictions with mean absolute percentage error <10%
        scores_df = scores_df[scores_df.mape < 0.1]

        # return predictions with least RMSE for each point/dq issue
        idx = scores_df.groupby('identifier')['rmse'].idxmin()
        scores_df = scores_df.loc[idx].reset_index(drop=True)

    return scores_df.drop(columns=["identifier"])

In [148]:
# ##################################################################
# # TESTING A NEW MODEL -------------------------------------------
# ##################################################################

# dictionary to save predictions for each point
# scores_df_dict = {
# "pointID": [],
# "predictions": [],
# "rmse": [],
# "mape": [],
# "modelName": []
# }
# # Create a DataFrame from the dictionary
# scores_df = pd.DataFrame(scores_df_dict)



# ## LOOP HERE @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# # for i, row in python_master_table.iterrows():
# i = 0
# row = master_table.iloc[i]


# #-----------------
# # INPUTS TO MODELS
# #-----------------
# # point id on SS
# pointID = row["pointID"]
# # data logging interval of the point 
# data_logging_interval = row["pointInterval"]
# freq = str(data_logging_interval.total_seconds()/3600)+"h"
# # the history df containing training data for the models
# df = row["his"]#.to_dataframe()                                      #### IMPORTANT : UNCOMMENT THIS ON SS  @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# df.set_index(df.columns[0], inplace=True, drop=True)
# df = df.asfreq(freq)
# ### FOR TESTING; creating a fake dqStart and fake length of missing data for testing
# # df.loc[df.index >= '2023-03-17', 'val'] = np.nan
# # dqStart = df[df.val.isna()].iloc[0].name
# # length_of_missing_data = df[df.val.isna()].iloc[-1].name - df[df.val.isna()].iloc[0].name
# ### FOR TESTING; keeping only the df.val column for testing
# # df = df[['val']]
# dqStart = row["dqStart"]
# length_of_missing_data = row["dqDuration"]


# # ----------------------
# # Training/Testing sets 
# # ----------------------
# data_before_gap = df.loc[:dqStart][:-1] # :-1 to not include the first ts with nulls, since df.loc[:dqStart] will include the first ts with NAN
# # doing a bfill and ffill to make sure there are no nulls in the training data. This step should be taken care of on SS by interpolating. This is done as a cautionary measure as nulls in the data will cause some models to fail
# data_before_gap.bfill(inplace=True)
# data_before_gap.ffill(inplace=True)
# train_data = data_before_gap.loc[:dqStart-length_of_missing_data][:-1]
# test_data = data_before_gap.loc[dqStart-length_of_missing_data:dqStart]
# # test_data timestamps used to slice the predictions_for_rmse df to have exact dimension as testing set. This prevents raising error due to mismatching lengths when using the rmse or mape functions
# test_data_timestamps = pd.date_range(start=test_data.index[0], end=test_data.index[-1], freq=test_data.index.freq)


# # ----------------------------
# # Timestamps for null duration
# # ----------------------------
# start = row['dqStart']
# duration = row['dqDuration']
# interval = row['pointInterval']
# timestamps = pd.date_range(start=start, end=start+duration, freq=interval)[1:-1] # clipping the first and last timestamps, as they already exist with actual data on SS


# #----------------------------
# # Dict of Data Quality Models                              ############# ADD NEW MODELS HERE 
# #----------------------------

# # UNIVARIATE Models
# dq_models_univariate = {
#     "Seasonal Naive" : seasonal_naive,
#     "Dynamic Optimized Theta": dynamic_optimized_theta
# }

# # MULTIVARIATE Models using one feature at a time
# dq_models_multivariate_1feature = {
#     "Polynomial Regression" : polynomial_regression,
# }

# # MULTIVARIATE Models using all features to predict target
# dq_models_multivariate = {
#     "Random Forest Regressor" : random_Forest_Regressor,
#     "KNN Regressor Uniform ": kNeighbors_Regressor_Uniform,
#     # "XGBoost 1": xgboost_1,
#     # "XGboost 2": xgboost_2,
#     # "XGboost 3": xgboost_3
# }



# ############################
# # LOOP FOR UNIVARIATE MODELS
# ############################
# ## loop here @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# # for model_name, model in dq_models_univariate.items():
# model_name = "Seasonal Naive"
# model = seasonal_naive


# #------------------------
# # ** Calculating RMSE **
# #------------------------
# # the prediction. USED ONLY TO EVALUATE RMSE
# predictions_for_rmse = model(data_before_gap, length_of_missing_data, data_logging_interval, dqStart-length_of_missing_data)
# predictions_for_rmse = predictions_for_rmse[predictions_for_rmse.index.isin(test_data_timestamps)]  
# rmse_score = mean_squared_error(test_data[test_data.columns[0]].to_numpy(), predictions_for_rmse[predictions_for_rmse.columns[0]].to_numpy(), squared=False)
# mape_score = mean_absolute_percentage_error(test_data[test_data.columns[0]].to_numpy(), predictions_for_rmse[predictions_for_rmse.columns[0]].to_numpy())

# #------------------
# # ** Predictions **
# #------------------
# # the predictions. USED FOR DATA CLEANING (uses all the data as training)
# predictions_for_data_quality = model(df, length_of_missing_data, data_logging_interval, dqStart)

# # keep only timestamps for null periods (rows where there are null values on SS)
# # predictions_for_data_quality = predictions_for_data_quality[predictions_for_data_quality.index.isin(timestamps)]   # UNCOMMENT THIS ONCE THE interval of the data column in SS master table is corrected

# # reset index to make the ts a column instead of index. SS doesnt show the index of a DF
# predictions_for_data_quality = predictions_for_data_quality.reset_index()

# # rename the ts and predictions column to "ts" and "predictions", to have similar naming for all ouutputs of models (makes it easier as well when using the dcInsert function on SS.)
# predictions_for_data_quality.columns = ["ts", "predictions"]

# # append data to the scores DF
# row_to_append = {'pointID': pointID, 'predictions': predictions_for_data_quality, 
#                 "rmse": rmse_score, "mape": mape_score,
#                 "modelName": model_name, 
#                 "identifier": 
#                     str(row["pointID"])
#                     +str(row["dqStart"])
#                     +str(row["dqDuration"])
#                     +str(row["dqType"])}

# scores_df = pd.concat([scores_df, pd.DataFrame([row_to_append])], ignore_index=True)

# scores_df

In [149]:
ens = ensemble_model(master_table)

c:\Users\abbayoumi\AppData\Local\anaconda3\envs\env_full\lib\site-packages\statsforecast\core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\abbayoumi\AppData\Local\anaconda3\envs\env_full\lib\site-packages\statsforecast\core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\abbayoumi\AppData\Local\anaconda3\envs\env_full\lib\site-packages\statsforecast\core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\abbayoumi\AppData\Local\anaconda3\envs\env_full\lib\sit

In [151]:
ens

,pointID,predictions,rmse,modelName,mape
0,@p:dmc_All:r:2e223a3a-5053d2b0 DMC Building 1 ...,ts predictions 0 ...,0.698491,Polynomial Regression - Feature: discharge_Fan...,0.027129
